# 01-Intro-Homework

Data: "Yellow Taxi Trip Records" for January and February 2023. Data will be present in the work directory

In [18]:
import pandas as pd
import pyarrow
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
# Read Parquet file
df_01 = pd.read_parquet("./yellow_tripdata_2023-01.parquet", engine="pyarrow")
df_02 = pd.read_parquet("./yellow_tripdata_2023-02.parquet", engine="pyarrow")

In [3]:
df_01.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
df_02.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


### Q1: Read the data for January. How many columns are there?

In [5]:
len(df_01.columns)

19

### Q2: Computing Duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.
What's the standard deviation of the trips duration in January?

In [6]:
df_01.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [7]:
df_01["tpep_dropoff_datetime"] = pd.to_datetime(df_01["tpep_dropoff_datetime"]) #finish
df_01["tpep_pickup_datetime"] = pd.to_datetime(df_01["tpep_pickup_datetime"]) #start
df_01["duration"] = df_01["tpep_dropoff_datetime"] - df_01["tpep_pickup_datetime"]
df_01["duration_seconds"] = df_01["duration"].dt.total_seconds()
std_duration = df_01["duration_seconds"].std()
print("Standard deviation (minutes):", std_duration/60.0)

Standard deviation (minutes): 42.594351241954534


### Q3: Dropping Outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).
**What fraction of the records left after you dropped the outliers?**

In [8]:
# Original number of records
total_records = len(df_01)

# Filter durations between 1 and 60 minutes (inclusive)
df_01["duration_minutes"] = df_01["duration_seconds"] / 60
df_filtered_01 = df_01[(df_01["duration_minutes"] >= 1) & (df_01["duration_minutes"] <= 60)]

filtered_records = len(df_filtered_01)

# Compute the fraction
fraction_left = filtered_records / total_records

print("Fraction of records left (%):", fraction_left*100)

Fraction of records left (%): 98.1220282212598


### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it
- **What's the dimensionality of this matrix (number of columns)?**

In [17]:
# Step 1: Convert relevant columns to string
df_filtered_01['PULocationID'] = df_filtered_01['PULocationID'].astype(str)
df_filtered_01['DOLocationID'] = df_filtered_01['DOLocationID'].astype(str)

# Step 2: Convert DataFrame to list of dictionaries
dicts = df_filtered_01[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Step 3: Fit DictVectorizer
dv = DictVectorizer()
X = dv.fit_transform(dicts)

# Step 4: Check dimensionality
X.shape

C:\Users\sprou\AppData\Local\Temp\ipykernel_34620\2795065154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_01['PULocationID'] = df_filtered_01['PULocationID'].astype(str)
C:\Users\sprou\AppData\Local\Temp\ipykernel_34620\2795065154.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_01['DOLocationID'] = df_filtered_01['DOLocationID'].astype(str)


(3009173, 515)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data
- **What's the RMSE on train?**

In [21]:
# Step 1: Set target variable
y = df_filtered_01['duration_minutes'].values

# Step 2: Train model
lr = LinearRegression()
lr.fit(X, y)

# Step 3: Predict
y_pred = lr.predict(X)

# Step 4: Compute RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f'RMSE on train: {rmse:.2f}')

RMSE on train: 7.65


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).
- **What's the RMSE on validation?**

In [23]:
## Computing duration in minutes for data in february
df_02["tpep_dropoff_datetime"] = pd.to_datetime(df_02["tpep_dropoff_datetime"]) #finish
df_02["tpep_pickup_datetime"] = pd.to_datetime(df_02["tpep_pickup_datetime"]) #start
df_02["duration"] = df_02["tpep_dropoff_datetime"] - df_02["tpep_pickup_datetime"]
df_02["duration_seconds"] = df_02["duration"].dt.total_seconds()
df_02["duration_minutes"] = df_02["duration_seconds"] / 60

In [24]:
# Original number of records
total_records = len(df_02)
# Filter durations between 1 and 60 minutes (inclusive)
df_filtered_02 = df_02[(df_02["duration_minutes"] >= 1) & (df_02["duration_minutes"] <= 60)]
filtered_records = len(df_filtered_02)
# Compute the fraction
fraction_left = filtered_records / total_records
print("Fraction of records left (%):", fraction_left*100)

Fraction of records left (%): 98.00944077722545


In [25]:
# Step 1: Make sure PULocationID and DOLocationID are strings
df_filtered_02.loc[:, 'PULocationID'] = df_filtered_02['PULocationID'].astype(str)
df_filtered_02.loc[:, 'DOLocationID'] = df_filtered_02['DOLocationID'].astype(str)

# Step 2: Convert to list of dictionaries
val_dicts = df_filtered_02[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Step 3: Transform using the fitted DictVectorizer
X_val = dv.transform(val_dicts)

# Step 4: Set target variable
y_val = df_filtered_02['duration_minutes'].values

# Step 5: Predict and compute RMSE
y_pred_val = lr.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f'RMSE on validation: {rmse_val:.2f}')

C:\Users\sprou\AppData\Local\Temp\ipykernel_34620\31436850.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['142' '132' '161' ... '158' '79' '161']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_filtered_02.loc[:, 'PULocationID'] = df_filtered_02['PULocationID'].astype(str)
C:\Users\sprou\AppData\Local\Temp\ipykernel_34620\31436850.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['163' '26' '145' ... '143' '162' '140']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_filtered_02.loc[:, 'DOLocationID'] = df_filtered_02['DOLocationID'].astype(str)


RMSE on validation: 7.81
